In [2]:
from bitarray import bitarray
import numpy as np

## Geração de chaves subjacentes

In [3]:
def permutation_p10(key):
    idexes = [2,4,1,6,3,9,0,8,7,5];
    key = np.array(key.tolist())
    key_p10 = key[idexes]
    return key_p10

example_key = bitarray('1010000010')
P10_example_key = permutation_p10(example_key)

print("K:     ", np.array(example_key.tolist()))
print("P10(K):", P10_example_key)

K:      [1 0 1 0 0 0 0 0 1 0]
P10(K): [1 0 0 0 0 0 1 1 0 0]


In [4]:
def subkeys_single_shift(key):
    key = np.array(key.tolist())
    key_left = key[:5]
    key_right = key[5:]
    key_left_shift = np.roll(key_left, -1)
    key_right_shift = np.roll(key_right, -1)
    return np.concatenate((key_left_shift, key_right_shift))

K_LR1_example = subkeys_single_shift(P10_example_key) 
print("P10(K):", P10_example_key)
print("K_LR1: ", K_LR1_example)

P10(K): [1 0 0 0 0 0 1 1 0 0]
K_LR1:  [0 0 0 0 1 1 1 0 0 0]


In [5]:
def permutation_p8(key):
    idexes = [5,2,6,3,7,4,9,8]
    key = np.array(key.tolist())
    key_p8 = key[idexes]
    return key_p8

print("K_LR1:    ", K_LR1_example)
P8_K1_example_key = permutation_p8(K_LR1_example)
print("P8(K_LR1):", P8_K1_example_key, "= K_1")



K_LR1:     [0 0 0 0 1 1 1 0 0 0]
P8(K_LR1): [1 0 1 0 0 1 0 0] = K_1


In [6]:
def subkeys_double_shift(key):
    key = np.array(key.tolist())
    key_left = key[:5]
    key_right = key[5:]
    key_left_shift = np.roll(key_left, -2)
    key_right_shift = np.roll(key_right, -2)
    return np.concatenate((key_left_shift, key_right_shift))

K_LR2_example = subkeys_double_shift(K_LR1_example)
P8_K2_example_key = permutation_p8(K_LR2_example)
print("K_LR1:    ", K_LR1_example)
print("K_LR2:    ", K_LR2_example)
print("P8(K_LR2):", P8_K2_example_key, "= K_2")

K_LR1:     [0 0 0 0 1 1 1 0 0 0]
K_LR2:     [0 0 1 0 0 0 0 0 1 1]
P8(K_LR2): [0 1 0 0 0 0 1 1] = K_2


## Permutação inicial $PI$

In [7]:
def initial_permutation(plaintext):
    idexes = [1,5,2,0,3,7,4,6]
    plaintext = np.array(plaintext.tolist())
    plaintext_ip = plaintext[idexes]
    return plaintext_ip

def inverse_initial_permutation(plaintext):
    idexes = [3,0,2,4,6,1,7,5]
    plaintext = np.array(plaintext.tolist())
    plaintext_ip = plaintext[idexes]
    return plaintext_ip


plaintext_example = bitarray('11010110')
IP_example = initial_permutation(plaintext_example)
print("P:           ", np.array(plaintext_example.tolist()))
print("IP(P):       ", IP_example)
print("IP^-1(IP(P)):", inverse_initial_permutation(IP_example))

P:            [1 1 0 1 0 1 1 0]
IP(P):        [1 1 0 1 1 0 0 1]
IP^-1(IP(P)): [1 1 0 1 0 1 1 0]


## Função F

In [8]:
def expansion_permutation(right_half):
    idexes = [3,0,1,2,1,2,3,0]
    right_half = np.array(right_half)
    right_half_ep = right_half[idexes]
    return right_half_ep

def S0(input):
    S0 = [
        [1,0,3,2],
        [3,2,1,0],
        [0,2,1,3],
        [3,1,3,2]
    ]
    # Aqui, há conversão manual de base binária para decimal
    row = int(input[0])*2 + int(input[3])
    column = int(input[1])*2 + int(input[2])
    binary = np.binary_repr(S0[row][column], 2)
    return bitarray(binary).tolist()

def S1(input):
    S1 = [
        [0,1,2,3],
        [2,0,1,3],
        [3,0,1,0],
        [2,1,0,3]
    ]
    row = int(input[0])*2 + int(input[3])
    column = int(input[1])*2 + int(input[2])
    binary = np.binary_repr(S1[row][column], 2)
    return bitarray(binary).tolist()

def P4(P4_input):
    idexes = [1,3,2,0]
    P4_input = np.array(P4_input)
    P4_output = P4_input[idexes]
    return P4_output

def F_function(half, subkey):
    half = half.tolist()
    subkey = subkey.tolist()
    
    half_ep = expansion_permutation(half)
    half_xor = np.logical_xor(half_ep, subkey)

    S0_input = half_xor[:4]
    S1_input = half_xor[4:]

    S0_S1 = np.concatenate([S0(S0_input), S1(S1_input)])

    return P4(S0_S1)

right_half_example = bitarray('1101')
subkey_example = bitarray('00100011')
print("R:      ", np.array(right_half_example.tolist()))
print("K:      ", np.array(subkey_example.tolist()))
print("f(R,K): ", F_function(right_half_example, subkey_example))

R:       [1 1 0 1]
K:       [0 0 1 0 0 0 1 1]
f(R,K):  [1 1 1 0]


## Função de substituição

In [9]:
def switch(plaintext):
    plaintext_left = plaintext[:4]
    plaintext_right = plaintext[4:]
    return np.concatenate((plaintext_right.tolist(), plaintext_left.tolist()))

print("SW: ", switch(bitarray('11000011')))


SW:  [0 0 1 1 1 1 0 0]


## Rodada de Feistel

In [10]:
def feistel_round(L, R, subkey):
    F_output = F_function(R, subkey)
    new_left = np.bitwise_xor(np.array(L.tolist()), F_output)
    cipher_text = np.concatenate((new_left, R.tolist()))
    return cipher_text

feistel_plaintext_example = bitarray('10111101')
print("f(L,R,K): ", feistel_round(feistel_plaintext_example[:4], 
                                feistel_plaintext_example[4:], 
                                subkey_example))

f(L,R,K):  [0 1 0 1 1 1 0 1]


## Trabalho

In [11]:
key = bitarray('1010000010')
plaintext = bitarray('11010111')

### Etapa 1: Gerar chaves subjacentes

In [12]:
K_arr = np.array(key.tolist())
print("K:", K_arr)

# Permutação P10 ➡️ Deslocamento circular ➡️ Permutação P8 ➡️ Deslocamento circular duplo ➡️ Permutação P8
K_P10 = permutation_p10(key)
K_LR1 = subkeys_single_shift(K_P10)
K1 = permutation_p8(K_LR1)
K_LR2 = subkeys_double_shift(K_LR1)
K2 = permutation_p8(K_LR2)

print("K1:", K1)
print("K2:", K2)

K: [1 0 1 0 0 0 0 0 1 0]
K1: [1 0 1 0 0 1 0 0]
K2: [0 1 0 0 0 0 1 1]


### Erapa 2: Permutação inicial $PI$

In [13]:
plaintext = initial_permutation(plaintext)
print('Blodo permutado: ', plaintext)

Blodo permutado:  [1 1 0 1 1 1 0 1]


### Etapa 3: Dividir em metades

In [14]:
L, R = plaintext[:4], plaintext[4:]
print('L:', L)
print('R:', R)

L: [1 1 0 1]
R: [1 1 0 1]


### Etapa 4: Rodadas de Feistel

In [15]:
C1 = feistel_round(L, R, K1)
print('C1 (antes do switch):', C1)

C1 = switch(C1)
print('C1:                  ', C1)

C1_L, C1_R = C1[:4], C1[4:]

C2 = feistel_round(C1_L, C1_R, K2)
print('C2:                  ', C2)

C1 (antes do switch): [0 0 1 0 1 1 0 1]
C1:                   [1 1 0 1 0 0 1 0]
C2:                   [0 0 1 1 0 0 1 0]


### Etapa 5: Permutação final

In [16]:
cipher_text = inverse_initial_permutation(C2)
print('Texto cifrado:', cipher_text)

Texto cifrado: [1 0 1 0 1 0 0 0]


### Extra: Sintetizando em uma única função

In [17]:
key = bitarray('1010000010')
plaintext = bitarray('11010111')

def SDES(key, plaintext):
    # Geração das subchaves
    K_P10 = permutation_p10(key)
    K_LR1 = subkeys_single_shift(K_P10)
    K1 = permutation_p8(K_LR1)
    K_LR2 = subkeys_double_shift(K_LR1)
    K2 = permutation_p8(K_LR2)

    # Permutação inicial
    plaintext = initial_permutation(plaintext)

    # Divisão em metades
    L, R = plaintext[:4], plaintext[4:]

    # Rodada 1
    C1 = feistel_round(L, R, K1)
    C1 = switch(C1)
    C1_L, C1_R = C1[:4], C1[4:]

    # Rodada 2
    C2 = feistel_round(C1_L, C1_R, K2)
    
    # Permutação final
    cipher_text = inverse_initial_permutation(C2)

    return cipher_text

print(SDES(key, plaintext))

[1 0 1 0 1 0 0 0]
